In [1]:
import digitalhub as dh
PROJECT_NAME = "deforestation"
proj = dh.get_or_create_project(PROJECT_NAME)

Register to the open data space copernicus(if not already) and get your credentials.

https://identity.dataspace.copernicus.eu/auth/realms/CDSE/login-actions/registration?client_id=cdse-public&tab_id=FIiRPJeoiX4

Log the credentials as project secret keys as shown below

In [7]:
# THIS NEED TO BE EXECUTED JUST ONCE
secret0 = proj.new_secret(name="CDSETOOL_ESA_USER", secret_value="esa_username")
secret1 = proj.new_secret(name="CDSETOOL_ESA_PASSWORD", secret_value="esa_password")

### Download data from Sentinel 2

In [ ]:
string_dict_data = """{
 "satelliteParams":{
     "satelliteType": "Sentinel2"
 },
 "startDate": "2018-01-01",
 "endDate": "2019-12-31",
 "geometry": "POLYGON((10.968432350469937 46.093829019481056,10.968432350469937 46.09650743619973, 10.97504139531014 46.09650743619973,10.97504139531014 46.093829019481056, 10.968432350469937 46.093829019481056))",
 "area_sampling": "true",
 "cloudCover": "[0,5]",
 "artifact_name": "data_s2_deforestation" 
 }"""

list_args =  ["main.py",string_dict_data]


In [ ]:
function_s2 = proj.new_function("download_images_s2",kind="container",image="ghcr.io/tn-aixpa/sentinel-tools:0.11.5",command="python")

In [ ]:
run = function_s2.run(
    action="job",
    secrets=["CDSETOOL_ESA_USER","CDSETOOL_ESA_PASSWORD"],
    fs_group='8877',
    args=["main.py", string_dict_data],
    resources={"cpu": {"requests": "3", "limits": "6"},"mem":{"requests": "32Gi", "limits": "64Gi"}},
    volumes=[{
        "volume_type": "persistent_volume_claim",
        "name": "volume-deforestation",
        "mount_path": "/app/files",
        "spec": {
             "size": "350Gi" 
        }
    }])

In [ ]:
run.refresh().status.state

In [3]:
artifact = proj.get_artifact('data')

### Perform Data Elaboration on RS-Tool

In [10]:
artifact_name='bosco'
src_path='bosco'
artifact_bosco = proj.log_artifact(name=artifact_name, kind="artifact", source=src_path)

In [4]:
# artifact_name='data'
# src_path='data'
# artifact_data = proj.log_artifact(name=artifact_name, kind="artifact", source=src_path)

In [8]:
function_rs = proj.new_function("elaborate",kind="container", image="ghcr.io/tn-aixpa/rs-deforestation:2.7_b4", command="/bin/bash", code_src="launch.sh")

In [ ]:
#function_rs = proj.get_function("elaborate")

In [ ]:
run_el = function_rs.run(
    action="job",
    fs_group='8877',
    resources={"cpu": {"requests": "6", "limits": "12"},"mem":{"requests": "32Gi", "limits": "64Gi"}},
    volumes=[{
        "volume_type": "persistent_volume_claim",
        "name": "volume-deforestation",
        "mount_path": "/app/files",
        "spec": { "size": "250Gi" }
    }],
    args=['/shared/launch.sh', 'bosco', 'data_s2_deforestation', '[2018,2019]',  'deforestation_2018_19']
)

In [11]:
run_el.refresh().status.state

'RUNNING'

### Pipeline

In [15]:
%%writefile "deforestation_pipeline.py"

from digitalhub_runtime_kfp.dsl import pipeline_context

def myhandler(startYear, endYear, geometry, shapeArtifactName, dataArtifactName, outputName):
    with pipeline_context() as pc:
        string_dict_data = """{"satelliteParams":{"satelliteType":"Sentinel2"},"startDate":\""""+ str(startYear) + """-01-01\","endDate": \"""" + str(endYear) + """-12-31\","geometry": \"""" + str(geometry) + """\","area_sampling":"true","cloudCover":"[0,5]","artifact_name": \"""" + str(dataArtifactName) + """\"}"""
        s1 = pc.step(name="download",
                     function="download_images_s2",
                     action="job",
                     secrets=["CDSETOOL_ESA_USER","CDSETOOL_ESA_PASSWORD"],
                     fs_group='8877',
                     args=["main.py", string_dict_data],
                     volumes=[{
                        "volume_type": "persistent_volume_claim",
                        "name": "volume-deforestation",
                        "mount_path": "/app/files",
                        "spec": { "size": "250Gi" }
                        }
                    ])
        s2 = pc.step(name="elaborate",
                     function="elaborate",
                     action="job",
                     fs_group='8877',
                     resources={"cpu": {"requests": "6", "limits": "12"},"mem":{"requests": "32Gi", "limits": "64Gi"}},
                     volumes=[{
                        "volume_type": "persistent_volume_claim",
                        "name": "volume-deforestation",
                        "mount_path": "/app/files",
                        "spec": { "size": "250Gi" }
                    }],
                     args=['/shared/launch.sh', str(shapeArtifactName), str(dataArtifactName), "[" + str(startYear) + ',' + str(endYear) + "]", str(outputName)]
                     ).after(s1)
     


Overwriting deforestation_pipeline.py


In [16]:
workflow = proj.new_workflow(name="pipeline_deforestation", kind="kfp", code_src= "deforestation_pipeline.py", handler = "myhandler")

In [17]:
wfbuild = workflow.run(action="build", wait=True)

2025-07-29 11:53:21,619 - INFO - Waiting for run ce47f8cd93244f23911de2dfa61e8775 to finish...
2025-07-29 11:53:26,640 - INFO - Waiting for run ce47f8cd93244f23911de2dfa61e8775 to finish...
2025-07-29 11:53:31,662 - INFO - Waiting for run ce47f8cd93244f23911de2dfa61e8775 to finish...
2025-07-29 11:53:36,685 - INFO - Waiting for run ce47f8cd93244f23911de2dfa61e8775 to finish...
2025-07-29 11:53:41,706 - INFO - Waiting for run ce47f8cd93244f23911de2dfa61e8775 to finish...
2025-07-29 11:53:46,729 - INFO - Run ce47f8cd93244f23911de2dfa61e8775 finished in 25.11 seconds.


### TPS

In [7]:
workflow_run = workflow.run(action="pipeline", parameters={
    "startYear": 2018,
    "endYear": 2019,
    "geometry": "POLYGON((10.968432350469937 46.093829019481056,10.968432350469937 46.09650743619973, 10.97504139531014 46.09650743619973,10.97504139531014 46.093829019481056, 10.968432350469937 46.093829019481056))",
    "shapeArtifactName":"bosco",
    'dataArtifactName': "data_s2_tps",
    "outputName": "deforestation_2018_19_tps"
    })

### TPR

In [18]:
workflow_run = workflow.run(action="pipeline", parameters={
    "startYear": 2018,
    "endYear": 2019,
    "geometry": "POLYGON((11.04350 45.87743, 11.04350 45.87943, 11.04550 45.87943, 11.04550 45.87743, 11.04350 45.87743))",
    "shapeArtifactName":"bosco",
    'dataArtifactName': "data_s2_tpr",
    "outputName": "deforestation_2018_19_tpr"
    })

### TQS

In [10]:
workflow_run = workflow.run(action="pipeline", parameters={
    "startYear": 2018,
    "endYear": 2019,
    "geometry": "POLYGON((11.90670 46.43071, 11.90670 46.43271, 11.90870 46.43271, 11.90870 46.43071, 11.90670 46.43071))",
    "shapeArtifactName":"bosco",
    'dataArtifactName': "data_s2_tqs",
    "outputName": "deforestation_2018_19_tqs"
    })